 # Keras Tutorial

Keras is a high-level neural networks API, written in Python and capable of running on top of certain lower-level frameworks like Tensorflow. It is one level above Tensorflow and aims at implementing the Deep Learning pipelines easily and quickly.

For our final project, we will be using Keras to build the entire pipeline from scratch. This includes data pre-processing, feature extraction (with CNN) and classification (with fully-connected nets). The goal of this tutorial is to give you adequate knowledge to prepare you for the final project.

What is good about Keras compared with other Deep Learning frameworks:
* It's high-level, which means that you can implement complex things with several lines of simple code
* It works directly with NumPy arrays, so you don't have to spend extra time on creating a Python class for dataset like PyTorch

Remember in the class we talked about the pipeline of a real computer vision system, in which we:

1. First clean the data to the format to be used for later steps (which includes data loading, data pre-processing, dataset splitting (we'll talk about this on Friday), data augmentation (which we're not gonna cover), etc);

2. Then we build the model for feature extraction as well as for final regression / classification. Remember we have many choices like linear model, fully connected neural nets, convolutional neural nets, etc. And we can implement these models very easily in Keras with just one line of code;

3. After we get the data and the model, we need to code up the optimization part (for which we'll use gradient descent). 

In this tutorial, we'll go over these parts sequentially.

## Data Loading and Pre-processing

So in Keras we don't need anything specific for data, we just use NumPy and represent our data in Numpy arrays. Now we're gonna create some fake data to be used later.

In [2]:
# Import necessary packages
import numpy as np

In [3]:
# Create random numpy arrays (ldata loading)
rand_data = np.random.random((1000, 32, 32, 3)) # We have 1000 fake images with spatial size 32 * 32
rand_label = np.array([0]*500 + [1]*500)        # Create fake binary labels for these images  

print(rand_data.shape)

(1000, 32, 32, 3)


In [4]:
# Split data into train, validation and test sets (we'll talk more about this on Friday)
train_ratio, val_ratio = 0.9, 0.05

X_train = rand_data[:int(rand_data.shape[0]*train_ratio), ...] # ... means all the other axes
y_train = rand_label[:int(rand_data.shape[0]*train_ratio), ...]

X_val = rand_data[int(rand_data.shape[0]*train_ratio):int(rand_data.shape[0]*(train_ratio+val_ratio)), ...]
y_val = rand_label[int(rand_data.shape[0]*train_ratio):int(rand_data.shape[0]*(train_ratio+val_ratio)), ...]

X_test = rand_data[int(rand_data.shape[0]*(train_ratio+val_ratio)):, ...]
y_test = rand_label[int(rand_data.shape[0]*(train_ratio+val_ratio)):, ...]

print(train_set.shape)
print(val_set.shape)
print(test_set.shape)

NameError: name 'train_set' is not defined

## Model construction

Now we have all the data, next we're gonna build our model for feature extraction as well as classification. In Keras, you can easily build many models, as shown below.

In [5]:
import keras
from keras.models import Sequential # Sequential is one of the main models in Keras, which is basically a sequentially stacked series of layers

model = Sequential() # Initialize a Sequential model instance

Using TensorFlow backend.
W0708 14:04:41.283819 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [6]:
# First we'll use fully-connected neural nets
from keras.layers import Dense # Dense is Keras's name for fully connected layers

# We can stack layers like lego blocks by simplying using `add()`
# `units` is the number of neurons
# `activation` is the nonlinear function we add for each layer
# We only need to specify `input_dim` which is the input dimension for the layer for the input layer, because for later layers the input is just the output from last layer
# Once again, the number of neurons in hidden layers (e.g., 64 and 16 here) are design choices

model.add(Dense(units=64, activation='sigmoid', input_dim=32*32*3)) 
model.add(Dense(units=16, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))

W0708 14:04:41.318618 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0708 14:04:41.325958 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [7]:
# Once the model is build, we then configure the learning process with `compile()`
# We need to specify the loss function, the optimizer and the metric we use to evaluate our model
# For loss here we're using a function called binary cross-entropy loss, which is specifically for binary classification
# For optimizer we're using gradient descent, which is written as 'sgd' in Keras
# Since we're doing classification, normally the classification accuracy is how we evaluate the model

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

W0708 14:04:41.388047 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0708 14:04:41.413199 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0708 14:04:41.420686 4359500736 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
# The above is actually a convenient way that Keras provides for easy implementation. If you want to have more control over the learning process (e.g., the learning rate), you can use the following:

model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=0.001))

In [9]:
# Up to this point we're all doing configurations. Now everything is set up so we're letting the model do real things!

# Since now we're using a fully-connected nets, remember we need to flatten the image to a single long vector first
X_train_flat = X_train.reshape((-1, 32*32*3)) # -1 means letting NumPy to figure this axis out automatically
X_val_flat = X_val.reshape((-1, 32*32*3))
X_test_flat = X_test.reshape((-1, 32*32*3))

print(X_train_flat.shape)
print(X_val_flat.shape)
print(X_test_flat.shape)

# Then use fit() to actually train our model
# epochs is basically how many iterations we want for the update process. The model needs some time to reach the optimal state!
# batch_size is how many images we use each time to estimate the gradient. Remember that the more we use the more accurate each update will be, but it will also be slower

model.fit(X_train_flat, y_train, epochs=5, batch_size=32, validation_data=(X_val_flat, y_val))

(900, 3072)
(50, 3072)
(50, 3072)


W0708 14:04:41.783230 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 900 samples, validate on 50 samples
Epoch 1/5
900/900 [==============================] - 0s 482us/step - loss: 0.7926 - val_loss: 1.4095
Epoch 2/5
900/900 [==============================] - 0s 117us/step - loss: 0.7733 - val_loss: 1.3433
Epoch 3/5
900/900 [==============================] - 0s 210us/step - loss: 0.7578 - val_loss: 1.2870
Epoch 4/5
900/900 [==============================] - 0s 118us/step - loss: 0.7455 - val_loss: 1.2367
Epoch 5/5
900/900 [==============================] - 0s 127us/step - loss: 0.7352 - val_loss: 1.1899


In [10]:
# Now let's see how our model does
acc = model.evaluate(X_test_flat, y_test)
print('The test accuracy is: {}'.format(acc))

# And make predictions
prob = model.predict(X_test_flat) # These are probabilities, and we want to convert them to class labels
label = np.array(prob > 0.5, dtype=int)

print('The predicted probabilities are: {}'.format(prob))
print('The predicted class labels are: {}'.format(label))


50/50 [==============================] - 0s 151us/step
The test accuracy is: 1.181574683189392
The predicted probabilities are: [[0.29468042]
 [0.32015872]
 [0.29702803]
 [0.2956799 ]
 [0.3032065 ]
 [0.3076725 ]
 [0.30662042]
 [0.31618002]
 [0.29678673]
 [0.31160414]
 [0.31556433]
 [0.3145309 ]
 [0.3043297 ]
 [0.29171878]
 [0.30827716]
 [0.31633997]
 [0.3225649 ]
 [0.30780837]
 [0.3049808 ]
 [0.30571368]
 [0.31296837]
 [0.3059591 ]
 [0.30577075]
 [0.2957374 ]
 [0.29871827]
 [0.31976348]
 [0.30585146]
 [0.2958573 ]
 [0.31252223]
 [0.30350444]
 [0.3080806 ]
 [0.30480707]
 [0.30357558]
 [0.30481586]
 [0.30484378]
 [0.30869654]
 [0.30685067]
 [0.29151616]
 [0.3149043 ]
 [0.31242874]
 [0.31596082]
 [0.32322848]
 [0.3065948 ]
 [0.28709796]
 [0.31754056]
 [0.31237417]
 [0.2991038 ]
 [0.32151175]
 [0.29705408]
 [0.3066637 ]]
The predicted class labels are: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]

In [11]:
# As we can expect, the results are totally random
# You can also play with other models, e.g., convnets
# So we do the same procedure once more

model = Sequential() # Re-initialize the model

# Feature extractor
# We're using such an architecture: conv -> maxpool -> conv -> maxpool
# 'same' padding means we zero-pad the images so that the output will be of the same size as the input
model.add(keras.layers.Conv2D(filters=16, kernel_size=3, strides=(2, 2), padding='same'))
model.add(keras.layers.Activation('sigmoid'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2))) # By default the stride is the same as the pooling size

model.add(keras.layers.Conv2D(filters=32, kernel_size=2, strides=(1, 1), padding='same'))
model.add(keras.layers.Activation('relu')) # ReLU is another kind of non-linear function
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

# Classifier
# We're using a 2-layer FC net for classification 
model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(32))
model.add(keras.layers.Activation('relu'))

model.add(keras.layers.Dense(1))
model.add(keras.layers.Activation('sigmoid'))

# Compilation
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.SGD(lr=0.001))

# Training
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluation
acc = model.evaluate(X_test, y_test)
print('The test accuracy is: {}'.format(acc))

# And make predictions
prob = model.predict(X_test) # These are probabilities, and we want to convert them to class labels
label = np.array(prob > 0.5, dtype=int)

print('The predicted probabilities are: {}'.format(prob))
print('The predicted class labels are: {}'.format(label))

W0708 14:04:42.912311 4359500736 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Train on 900 samples, validate on 50 samples
Epoch 1/10
900/900 [==============================] - 1s 751us/step - loss: 0.6926 - val_loss: 0.7170
Epoch 2/10
900/900 [==============================] - 0s 239us/step - loss: 0.6901 - val_loss: 0.7521
Epoch 3/10
900/900 [==============================] - 0s 219us/step - loss: 0.6885 - val_loss: 0.7710
Epoch 4/10
900/900 [==============================] - 0s 242us/step - loss: 0.6879 - val_loss: 0.7934
Epoch 5/10
900/900 [==============================] - 0s 244us/step - loss: 0.6876 - val_loss: 0.7961
Epoch 6/10
900/900 [==============================] - 0s 258us/step - loss: 0.6876 - val_loss: 0.7983
Epoch 7/10
900/900 [==============================] - 0s 259us/step - loss: 0.6875 - val_loss: 0.8010
Epoch 8/10
900/900 [==============================] - 0s 228us/step - loss: 0.6874 - val_loss: 0.8116
Epoch 9/10
900/900 [==============================] - 0s 347us/step - loss: 0.6875 - val_loss: 0.8033
Epoch 10/10
50/50 [==================